In [1]:
# 기본 import template
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 (필요시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

In [2]:
# data load
ds_sst = xr.open_dataset('../data/raw/oisst/oisst_monthly_2015_2024_kuroshio.nc')
ds_mld = xr.open_dataset('../data/raw/glorys/glorys_mld_2015_2024_kuroshio.nc')

# 영역 좌표 정의
LON_MIN, LON_MAX = 115, 160
LAT_MIN, LAT_MAX = 20, 50

In [3]:
# coords
print(list(ds_sst.coords))
print(list(ds_mld.coords))

['lon', 'lat', 'time']
['longitude', 'latitude', 'time']


In [4]:
ds_mld = ds_mld.rename({
    'longitude' : 'lon',
    'latitude' : 'lat'
})
print(ds_mld.coords)

Coordinates:
  * lon      (lon) float32 2kB 115.0 115.1 115.2 115.2 ... 159.8 159.9 160.0
  * lat      (lat) float32 1kB 20.0 20.08 20.17 20.25 ... 49.75 49.83 49.92 50.0
  * time     (time) datetime64[ns] 960B 2015-01-01 2015-02-01 ... 2024-12-01


---

In [5]:
# 경로
from pathlib import Path
outdir = Path("../data/processed")
outdir.mkdir(parents=True, exist_ok=True)

### 1. 시간 교집합 맞추기

In [7]:
print(type(ds_sst.time.min().item()), ds_sst.time.dtype)
print(type(ds_mld.time.min().item()), ds_mld.time.dtype)
print(t0, type(t0))

<class 'int'> datetime64[ns]
<class 'int'> datetime64[ns]
1420070400000000000 <class 'int'>


In [9]:
t0 = np.datetime64(max(ds_sst.time.min().values, ds_mld.time.min().values)) 
t1 = np.datetime64(min(ds_sst.time.max().values, ds_mld.time.max().values))

ds_sst2 = ds_sst.sel(time = slice(t0, t1)).sortby('time')   
ds_mld2 = ds_mld.sel(time = slice(t0, t1)).sortby('time')

In [10]:
print(ds_sst.time.min().values, type(ds_sst.time.min().values))
print(ds_sst.time.to_index().__class__)


2015-01-01T00:00:00.000000000 <class 'numpy.datetime64'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


### A. Interp: GLORYS MLD -> SST 격자
- 빠르게 분석하는 용도

### B. xESMF regriding : 격자 평균에 가까운 방식
- 고해상도 정보를 저해상도 격자 셀로 면적 평균에 가깝게 내리기
- MLD같은 경우 `conservative_normed` 활용